In [13]:
import os
import sys
sys.path.append('../')
import pickle
import pandas as pd
import numpy as np

# Evaluation of CF Recommender Systems

In [14]:
# TEST SET
df_test = pd.read_pickle("files/testset.pkl")[["reviewerID", "asin", "overall"]]\
            .rename(columns={"reviewerID": "uid", "asin": "iid"})

# PREDICTIONS
nb = pd.read_pickle("files/preds_knn.pkl")
lf = pd.read_pickle("files/preds_svd.pkl")
pred_nb_list = list(nb.itertuples(index=False))
pred_lf_list = list(lf.itertuples(index=False))

# Detect users from training set that are not in test
nb_users = set([pred.uid for pred in pred_nb_list])
lf_users = set([pred.uid for pred in pred_lf_list])
nb_users_in_pred_but_not_in_test = list(nb_users.difference(set(df_test['uid'])))
lf_users_in_pred_but_not_in_test = list(lf_users.difference(set(df_test['uid'])))
assert nb_users_in_pred_but_not_in_test == lf_users_in_pred_but_not_in_test
print(f"There are {len(lf_users_in_pred_but_not_in_test)} users in the training set that are not in the test set.")

# Remove these users' predictions for evaluation
print("Lengths before removing preds not in test set:", len(nb), len(lf))
nb = nb[~nb.uid.isin(nb_users_in_pred_but_not_in_test)]
lf = lf[~lf.uid.isin(nb_users_in_pred_but_not_in_test)]
print("After removing:", len(nb), len(lf))

nb_merge = nb.merge(df_test, how="inner", on=["uid", "iid"])
print("\nkNN RMSE:", np.sqrt(np.mean((nb_merge["overall"] - nb_merge["score"])**2)))

lf_merge = lf.merge(df_test, how="inner", on=["uid", "iid"])
print("SVD RMSE:", np.sqrt(np.mean((lf_merge["overall"] - lf_merge["score"])**2)))

There are 113 users in the training set that are not in the test set.
Lengths before removing preds not in test set: 1449029 1449029
After removing: 1359246 1359246

kNN RMSE: 1.1497250696788386
SVD RMSE: 0.9972278662314285


In [16]:
# often used functions (see metrics.py; rather concise implementations)
from metrics import PatK, MAPatK, MRRatK, HRatK

ks = [5, 15]

print(12*" " + "NB | LF")

for k in ks:
    P_nb, P_lf = PatK(nb, df_test, k), PatK(lf, df_test, k)
    MAP_nb, MAP_lf = MAPatK(nb, df_test, k), MAPatK(lf, df_test, k)
    MRR_nb, MRR_lf = MRRatK(nb, df_test, k), MRRatK(lf, df_test, k)
    MHR_nb, MHR_lf = HRatK(nb, df_test, k), HRatK(lf, df_test, k)
    print(f"  P@{k:2g} = {P_nb  :.4f}|{P_lf  :.4f}")
    print(f"MAP@{k:2g} = {MAP_nb:.4f}|{MAP_lf:.4f}")
    print(f"MRR@{k:2g} = {MRR_nb:.4f}|{MRR_lf:.4f}")
    print(f" HR@{k:2g} = {MHR_nb:.4f}|{MHR_lf:.4f}\n") 


            NB | LF
  P@ 5 = 0.0023|0.0153
MAP@ 5 = 0.0070|0.0172
MRR@ 5 = 0.0070|0.0172
 HR@ 5 = 0.0117|0.0766

  P@15 = 0.0016|0.0092
MAP@15 = 0.0085|0.0264
MRR@15 = 0.0085|0.0264
 HR@15 = 0.0245|0.1373



## Error analysis

In [ ]:
dft = pd.read_pickle("testset.pkl")[["reviewerID", "asin", "overall", "unixReviewTime"]] \
        .sort_values("unixReviewTime") \
        .reset_index(drop=True)

uid_first, uid_last = dft.iloc[[0,-1]]["reviewerID"]
uid_first, uid_last


## Excercise 4

Based on the top-5, top-10 and top-20 predictions from Exercise 2, compute the systems’ hit rate averaged over the total number of users in the test set.

In [ ]:
merged = relevant_column(nb, df_test, 5)
scores = merged[["uid", "iid", "relevant"]].groupby(by="uid")["relevant"].apply(lambda x: x.any()*1)
scores.mean()

In [ ]:
ks = [5, 10, 20]

print(12*" " + "NB | LF")

for k in ks:
    MHR_nb, MHR_lf = HRatK(nb, df_test, k), HRatK(lf, df_test, k)
    print(f"MHR@{k:2g} = {MHR_nb:.4f}|{MHR_lf:.4f}")

In [ ]:
df_train = pd.read_pickle("train.pkl")
df = df_train[["overall", "reviewerID", "asin"]]

df_with_nans = df.pivot_table(values="overall", index="reviewerID", columns="asin")
df = df_with_nans.fillna(0)

corrs = df.T.corr("pearson")

In [ ]:
first_nbs = corrs.loc[uid_first].nlargest(11)[1:]
last_nbs = corrs.loc[uid_last].nlargest(11)[1:]
first_nbs

In [ ]:
reviewed_asins_first

In [ ]:
reviewed_asins_first = df_train[df_train["reviewerID"]==uid_first]["asin"]
reviewed_asins_last = df_train[df_train["reviewerID"]==uid_last]["asin"]

In [ ]:
nb_reviews = []
for rid in first_nbs.index: 
    nb_reviews.append(df_train[df_train["reviewerID"]==rid][["overall", "reviewerID", "asin"]])
pd.concat(nb_reviews)

In [ ]:
dff = df_train[df_train["reviewerID"]==uid_first]
dff["count"] = df_train[df_train["asin"].isin(reviewed_asins_first)].groupby("asin")["overall"].count()
dff

In [ ]:
df_train[df_train["asin"].isin(reviewed_asins_last)].groupby("asin")["overall"].count()

In [ ]:
df_train[df_train["asin"]=='B0014KJ6EQ']